In [ ]:
# A new environment is recommended:
# conda create -n test_env python numpy h5py xarray rioxarray netCDF4 jupyter matplotlib pandas
# conda activate test_env
# pip install pvlib windpowerlib

# Test of daily PV power production map
## 1. read one-year climate data

In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import datetime
import re
import gc
import os
import xarray as xr

# set 
scenario = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
climate_models = 'ACCESS-ESM1-5'            
s = 2
year = 2015

#
# Read the variables
#
t = datetime.datetime.now()
tasmax = xr.open_dataset( './data/tasmax_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc' )['tasmax']
tasmax.coords['lon'] = (tasmax.coords['lon'] + 180) % 360 - 180
tasmax = tasmax.sortby(tasmax.lon) 
tasmax = tasmax.sortby(tasmax.lat, ascending = False) 



tasmin = xr.open_dataset('./data/tasmin_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc')['tasmin'] 
tasmin.coords['lon'] = (tasmin.coords['lon'] + 180) % 360 - 180
tasmin = tasmin.sortby(tasmin.lon) 
tasmin = tasmin.sortby(tasmin.lat, ascending = False) 



tas = xr.open_dataset('./data/tas_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc')['tas']   
tas.coords['lon'] = (tas.coords['lon'] + 180) % 360 - 180
tas = tas.sortby(tas.lon) 
tas = tas.sortby(tas.lat, ascending = False) 



rsds = xr.open_dataset( './data/rsds_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc')['rsds'] 
rsds.coords['lon'] = (rsds.coords['lon'] + 180) % 360 - 180
rsds = rsds.sortby(rsds.lon) 
rsds = rsds.sortby(rsds.lat, ascending = False) 
rsds.values[rsds.values<0] = 0
rsds.values[rsds.values>600] = 600


wind_speed = xr.open_dataset('./data/sfcWind_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc')['sfcWind'] 
wind_speed.coords['lon'] = (wind_speed.coords['lon'] + 180) % 360 - 180
wind_speed = wind_speed.sortby(wind_speed.lon) 
wind_speed = wind_speed.sortby(wind_speed.lat, ascending = False)   


# skip ocean
pixel_type = np.full((len(tasmax.lat), len(tasmax.lon)), 0, dtype = np.int8)
pixel_type[~np.isnan(tasmax[0,:])] = 1


# Extract length info from one file
days = wind_speed.shape[0]
size = wind_speed.shape[1] * wind_speed.shape[2]

# Extract lon/lat info from one file
lat = wind_speed.lat.values
lon = wind_speed.lon.values
lons, lats = np.meshgrid(lon,lat) 
time = wind_speed.time.values

years = [year for i in range(size)]

quick_pmp = [1 for i in range(size)] 
hourly_output = [0 for i in range(size)] 


# combine
# year, rs, temp_mean, temp_min, temp_max, wind_speed, lon, lat, 
input_para = np.array(list(zip(pixel_type.reshape(-1).T, 
                       years, 
                       rsds.values.reshape(rsds.shape[0],-1).T, 
                       wind_speed.values.reshape(wind_speed.shape[0],-1).T, 
                       tas.values.reshape(tas.shape[0],-1).T, 
                       tasmin.values.reshape(tasmin.shape[0],-1).T, 
                       tasmax.values.reshape(tasmax.shape[0],-1).T,                         
                       lons.reshape(-1).T, 
                       lats.reshape(-1).T,
                       quick_pmp,
                       hourly_output    
                   )), dtype=object)


gc.collect()    
print(year, 'finished: combine data', datetime.datetime.now()-t)

2015 finished: combine data 0:01:05.611233


## 2. calculate global pv power map

In [2]:
import multiprocessing  
import pv_power_function_2025
def multi_process(prim):
    pool =multiprocessing.Pool(multiprocessing.cpu_count())
    results=pool.map( pv_power_function_2025.main, prim)
    out = list(results)
    return out

# run_global
t = datetime.datetime.now()
r = multi_process(input_para)
print(datetime.datetime.now() - t )

9:22:49.955887


In [3]:
maps = np.array(r, dtype = np.float32).reshape(len(lat), len(lon), len(time))   
maps = xr.DataArray(maps, dims = ['lat','lon','time'], name = 'pv_power_production',
                    coords={'lat': lat,'lon': lon, 'time':time})
maps.attrs['units'] = 'Wh/m2'  

if not os.path.exists('output'):
    os.makedirs('output')
    
maps.to_netcdf(r'./output/demo_global_pv_power_' + scenario[s] + '_' + str(year) + '_' + climate_models +'.nc', 
               encoding={'pv_power_production': {'zlib': True, 'complevel': 6}}) 

print('finished output', climate_models, scenario[s], year)

finished output ACCESS-ESM1-5 ssp245 2015


# Test of daily wind power production map
## 1. read one-year climate data

In [4]:
import numpy as np
import datetime
import re
import gc
import os
import rioxarray
import xarray as xr

# set 
scenario = ['historical', 'ssp126', 'ssp245', 'ssp370', 'ssp585']
climate_models = 'ACCESS-ESM1-5'            
s = 2
year = 2015

#
# Read the variables
#
t = datetime.datetime.now()
tas = xr.open_dataset('./data/tas_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc')['tas']   
tas.coords['lon'] = (tas.coords['lon'] + 180) % 360 - 180
tas = tas.sortby(tas.lon) 
tas = tas.sortby(tas.lat, ascending = False) 


wind_speed = xr.open_dataset('./data/sfcWind_day_ACCESS-ESM1-5_ssp245_r1i1p1f1_gn_2015.nc')['sfcWind'] 
wind_speed.coords['lon'] = (wind_speed.coords['lon'] + 180) % 360 - 180
wind_speed = wind_speed.sortby(wind_speed.lon) 
wind_speed = wind_speed.sortby(wind_speed.lat, ascending = False)   


# DEM
dem_map = rioxarray.open_rasterio('./data/dem.tif')[0,:]
dem_map.coords['x'] = (dem_map.coords['x'] + 180) % 360 - 180
dem_map = dem_map.sortby(dem_map.x) 
dem_map = dem_map.sel(y = slice(89.75, -60))
dem_map = dem_map.sortby(dem_map.y, ascending = False) 

#
# read power law exponent 
alpha_map = xr.open_dataset('./data/alpha_mean_season_1985-2014.nc')['alpha']
alpha_map = alpha_map.isel(lat=alpha_map.lat > -60)
alpha_map = alpha_map.sel(season = wind_speed["time"].dt.season)

pixel_type = np.full((len(wind_speed.lat), len(wind_speed.lon)), 0, dtype = np.int8)
pixel_type[~np.isnan(wind_speed[0,:])] = 1

# combine
input_para = np.array(list(zip(pixel_type.reshape(-1).T, 
                               tas.values.reshape(tas.shape[0],-1).T, 
                               dem_map.values.reshape(-1).T, 
                               wind_speed.values.reshape(wind_speed.shape[0],-1).T, 
                               alpha_map.values.reshape(alpha_map.shape[0],-1).T)), 
                      dtype = object)

print(year, 'finished: combine data', datetime.datetime.now()-t)

2015 finished: combine data 0:00:50.843596


## 2. calculate global wind power map

In [5]:
import multiprocessing  
import wind_power_function_2025
def multi_process(prim):
    pool =multiprocessing.Pool(multiprocessing.cpu_count())
    results=pool.map( wind_power_function_2025.main, prim)
    out = list(results)
    return out
    

# run_global
t = datetime.datetime.now()
r = multi_process(input_para)
print(datetime.datetime.now() - t )

0:35:32.021144


In [6]:
maps = np.array(r, dtype = np.float32).reshape(wind_speed.shape[1], wind_speed.shape[2], days)

lon = wind_speed.lon.values
lat = wind_speed.lat.values
time = wind_speed.time.values
maps = xr.DataArray(maps, dims = ['lat','lon','time'], name = 'wind_power_production',
                    coords={'lat': lat,'lon': lon, 'time':time})
maps.attrs['units'] = 'W'   

if not os.path.exists('output'):
    os.makedirs('output')

maps.to_netcdf(r'./output/demo_global_wind_power_' + scenario[s] + '_' + str(year) + '_' + climate_models +'.nc', 
               encoding={'wind_power_production': {'zlib': True, 'complevel': 6}}) 

print('finished output', climate_models, scenario[s], year)

finished output ACCESS-ESM1-5 ssp245 2015
